In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import re

#pd.set_option('display.max_columns', None)  # or 1000
#pd.set_option('display.max_rows', None)  # or 1000
#pd.set_option('display.max_colwidth', None)  # or 199

In [4]:
# Call the API
def get_response(url):
    response = requests.request("GET", url).text
    return json.loads(response)

In [5]:
#Import History Data First:
url_hist = "https://api.rootnet.in/covid19-in/stats/history"
url_latest = "https://api.rootnet.in/covid19-in/stats/latest"
conn = sqlite3.connect('C:/Users/Punam-Manu/Desktop/covid_data.sqlite')


In [6]:
# Get History data
response_info_hist = get_response(url_hist)
day_data_hist = []
for day_data in response_info_hist['data']:
    for covid_data_hist in day_data['regional']:
        day_data_hist.append(['India',covid_data_hist['loc'],covid_data_hist['totalConfirmed'],covid_data_hist['discharged'],covid_data_hist['deaths']
                       ,(covid_data_hist['totalConfirmed']-(covid_data_hist['discharged'] + covid_data_hist['deaths'])),day_data['day'] + ' 00:00:00'])

In [7]:
# Get Current Data
response_info = get_response(url_latest)
now = datetime.now().strftime("%Y-%m-%d") + " 00:00:00"
covid_data = []

for covid_state in response_info['data']['regional']:
    covid_data.append(['India',covid_state['loc'],covid_state['totalConfirmed'],covid_state['discharged'],covid_state['deaths']
                       ,(covid_state['totalConfirmed']-(covid_state['discharged'] + covid_state['deaths'])),now])


In [8]:
covid_df_history = pd.DataFrame(data=day_data_hist, columns = ['Country','State','Total_Cases','Discharged','Deaths','Active_Cases','Date'])
covid_df = pd.DataFrame(data=covid_data, columns = ['Country','State','Total_Cases','Discharged','Deaths','Active_Cases','Date'])

In [9]:
#covid_df_history
#covid_df
covid_df_all = pd.concat([covid_df_history,covid_df],ignore_index=True)

In [10]:
covid_df_all = covid_df_all[~covid_df_all['State'].str.contains(r'[@#&$%+-/*]')]
covid_df_all = covid_df_all[(covid_df_all['State'] != "Dadar Nagar Haveli") & (covid_df_all['State'] != "Telengana")]

# Buld a logic for calculating one day rise in all segment
covid_df_all['Change_in_Active'] = covid_df_all.groupby(['Country','State'])['Active_Cases'].shift(1,fill_value = 0)
covid_df_all['Change_in_Active'] = covid_df_all['Active_Cases'] - covid_df_all['Change_in_Active']

covid_df_all['Change_in_Discharged'] = covid_df_all.groupby(['Country','State'])['Discharged'].shift(1,fill_value = 0)
covid_df_all['Change_in_Discharged'] = covid_df_all['Discharged'] - covid_df_all['Change_in_Discharged']

covid_df_all['Change_in_Deaths'] = covid_df_all.groupby(['Country','State'])['Deaths'].shift(1,fill_value = 0)
covid_df_all['Change_in_Deaths'] = covid_df_all['Deaths'] - covid_df_all['Change_in_Deaths']

covid_df_all.to_sql('covid_data', conn, if_exists='replace', index=False)

#covid_df_all

In [11]:
covid_df_all

,Country,State,Total_Cases,Discharged,Deaths,Active_Cases,Date,Change_in_Active,Change_in_Discharged,Change_in_Deaths
0,India,Delhi,4,0,0,4,2020-03-10 00:00:00,4,0,0
1,India,Haryana,14,0,0,14,2020-03-10 00:00:00,14,0,0
2,India,Karnataka,1,0,0,1,2020-03-10 00:00:00,1,0,0
3,India,Kerala,9,0,0,9,2020-03-10 00:00:00,9,0,0
4,India,Maharashtra,2,0,0,2,2020-03-10 00:00:00,2,0,0
...,...,...,...,...,...,...,...,...,...,...
15751,India,Telangana,591170,560776,3378,27016,2021-06-07 00:00:00,-2192,3614,14
15752,India,Tripura,55469,48984,564,5921,2021-06-07 00:00:00,-560,789,6
15753,India,Uttarakhand,334024,311200,6699,16125,2021-06-07 00:00:00,-1180,1591,35
15754,India,Uttar Pradesh,1698389,1659209,21236,17944,2021-06-07 00:00:00,-1494,2446,85


In [25]:
import datetime

data = pd.read_sql("Select * from covid_data", conn)
mask = (data.State == 'Assam')
filtered_data = data.loc[mask, :].reset_index(drop=True)
start_date = datetime.date.today().strftime("%Y-%m-%d")
end_date=(datetime.datetime.now() - datetime.timedelta(30)).strftime("%Y-%m-%d")
end_date

'2021-04-15'

In [36]:
new_data = filtered_data[(filtered_data['Date'] < start_date) &(filtered_data['Date'] > end_date)]
new_data

,Country,State,Total_Cases,Discharged,Deaths,Active_Cases,Date,Change_in_Active,Change_in_Discharged,Change_in_Deaths
379,India,Assam,221868,216199,1122,4547,2021-04-15 00:00:00,276,106,3
380,India,Assam,222367,216284,1123,4960,2021-04-16 00:00:00,413,85,1
381,India,Assam,222940,216397,1127,5416,2021-04-17 00:00:00,456,113,4
382,India,Assam,223816,216546,1129,6141,2021-04-18 00:00:00,725,149,2
383,India,Assam,224455,216705,1135,6615,2021-04-19 00:00:00,474,159,6
384,India,Assam,225822,217017,1142,7663,2021-04-20 00:00:00,1048,312,7
385,India,Assam,227473,217296,1145,9032,2021-04-21 00:00:00,1369,279,3
386,India,Assam,229138,217593,1150,10395,2021-04-22 00:00:00,1363,297,5
387,India,Assam,231069,217991,1160,11918,2021-04-23 00:00:00,1523,398,10
388,India,Assam,233453,218339,1172,13942,2021-04-24 00:00:00,2024,348,12


In [12]:
data = pd.read_sql("Select * from covid_data", conn)
data

,Country,State,Total_Cases,Discharged,Deaths,Active_Cases,Date,Change_in_Active,Change_in_Discharged,Change_in_Deaths
0,India,Delhi,4,0,0,4,2020-03-10 00:00:00,4,0,0
1,India,Haryana,14,0,0,14,2020-03-10 00:00:00,14,0,0
2,India,Karnataka,1,0,0,1,2020-03-10 00:00:00,1,0,0
3,India,Kerala,9,0,0,9,2020-03-10 00:00:00,9,0,0
4,India,Maharashtra,2,0,0,2,2020-03-10 00:00:00,2,0,0
...,...,...,...,...,...,...,...,...,...,...
14508,India,Telangana,525007,469007,2928,53072,2021-05-16 00:00:00,-3845,12387,61
14509,India,Tripura,40816,35522,431,4863,2021-05-16 00:00:00,829,376,5
14510,India,Uttarakhand,283239,198616,4623,80000,2021-05-16 00:00:00,1696,9355,378
14511,India,Uttar Pradesh,1609140,1414259,17238,177643,2021-05-16 00:00:00,-27015,54583,592


In [20]:
rise_active

,Change_in_Active,Country,State,Total_Cases,Discharged,Deaths,Active_Cases,Date,Change_in_Active,Change_in_Discharged,Change_in_Deaths
0,18,India,Andaman and Nicobar Islands,6542,6225,85,232,2021-05-15 00:00:00,18,12,2
1,2745,India,Andhra Pradesh,1388803,1175843,9173,203787,2021-05-15 00:00:00,2745,19177,96
2,35,India,Arunachal Pradesh,21373,19094,77,2202,2021-05-15 00:00:00,35,223,5
3,-166,India,Assam,319632,274247,2060,43325,2021-05-15 00:00:00,-166,4168,76
4,-6714,India,Bihar,637679,544445,3670,89564,2021-05-15 00:00:00,-6714,14131,77
5,-283,India,Chandigarh,54043,45268,617,8158,2021-05-15 00:00:00,-283,925,8
6,-3486,India,Chhattisgarh,899925,772500,11461,115964,2021-05-15 00:00:00,-3486,10908,172
7,-129,India,Dadra and Nagar Haveli and Daman and Diu,9335,8404,4,927,2021-05-15 00:00:00,-129,207,0
8,-5923,India,Delhi,1380981,1288280,20907,71794,2021-05-15 00:00:00,-5923,14140,289
9,-566,India,Goa,132585,98200,1998,32387,2021-05-15 00:00:00,-566,2960,61
